In [6]:
import mysql.connector
import pandas as pd

connection = mysql.connector.connect(
    host="localhost",
    user="python_user",
    password="aizen",
    database="game_recommender"
)
query = "SELECT name, genre, short_des FROM popular_games ;"
df = pd.read_sql(query, connection)
print(df)
connection.close()


                          name  \
0               Counter-Strike   
1        Team Fortress Classic   
2                Day of Defeat   
3           Deathmatch Classic   
4    Half-Life: Opposing Force   
..                         ...   
127       Monster Hunter Wilds   
128         Black Myth: Wukong   
129            Path of Exile 2   
130                 Schedule I   
131                   R.E.P.O.   

                                                 genre  \
0                                               Action   
1                                               Action   
2                                               Action   
3                                               Action   
4                                               Action   
..                                                 ...   
127                             Action, Adventure, RPG   
128                             Action, Adventure, RPG   
129  Action, Adventure, Massively Multiplayer, RPG,...   
130  Action

C:\Users\Yash\AppData\Local\Temp\ipykernel_16076\1334102322.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [3]:
import aiohttp
import asyncio
import mysql.connector
from tqdm import tqdm
import nest_asyncio

# Allow asyncio to run in Jupyter/VS Code
nest_asyncio.apply()

# --------------------------------------------
# 1️⃣ Connect to MySQL Database
# --------------------------------------------
connection = mysql.connector.connect(
    host="localhost",
    user="python_user",
    password="aizen",
    database="game_recommender"
)
cursor = connection.cursor()

# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS popular_games (
    appid INT PRIMARY KEY,
    name TEXT,
    genre TEXT,
    categories TEXT,
    short_des TEXT,
    header_image TEXT,
    developer TEXT,
    publisher TEXT,
    release_date TEXT
)
""")
connection.commit()

# --------------------------------------------
# 2️⃣ Fetch All Steam Apps
# --------------------------------------------
async def fetch_all_apps():
    url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as res:
            data = await res.json()
            return data["applist"]["apps"]

# --------------------------------------------
# 3️⃣ Fetch App Details (Concurrent)
# --------------------------------------------
async def fetch_app_details(session, app, sem):
    appid = app["appid"]
    name = app["name"]

    async with sem:  # Limit concurrency
        try:
            url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
            async with session.get(url, timeout=8) as res:
                data = await res.json()
                app_data = data.get(str(appid), {}).get("data", None)

                # Skip empty/unreleased games
                if not app_data or not app_data.get("release_date", {}).get("date"):
                    return None

                genres = ", ".join([g["description"] for g in app_data.get("genres", [])]) if app_data.get("genres") else ""
                categories = ", ".join([c["description"] for c in app_data.get("categories", [])]) if app_data.get("categories") else ""
                short_des = app_data.get("short_description", "")
                header_image = app_data.get("header_image", "")
                developer = ", ".join(app_data.get("developers", [])) if app_data.get("developers") else ""
                publisher = ", ".join(app_data.get("publishers", [])) if app_data.get("publishers") else ""
                release_date = app_data.get("release_date", {}).get("date", "")

                return (appid, name, genres, categories, short_des, header_image, developer, publisher, release_date)

        except Exception:
            return None

# --------------------------------------------
# 4️⃣ Main Fetcher Function
# --------------------------------------------
async def main(limit=5000):
    apps = await fetch_all_apps()
    print(f"✅ Total apps fetched from Steam: {len(apps)}")

    # Limit to top few thousand (fast + popular enough)
    apps = apps[:limit]

    sem = asyncio.Semaphore(40)  # 40 concurrent requests
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_app_details(session, app, sem) for app in apps]
        results = []

        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Fetching games"):
            result = await f
            if result:
                results.append(result)

    # --------------------------------------------
    # 5️⃣ Insert into MySQL
    # --------------------------------------------
    print(f"💾 Inserting {len(results)} valid games into MySQL...")
    insert_query = """
    INSERT INTO popular_games (appid, name, genre, categories, short_des, header_image, developer, publisher, release_date)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
    ON DUPLICATE KEY UPDATE
    name=VALUES(name), genre=VALUES(genre), categories=VALUES(categories),
    short_des=VALUES(short_des), header_image=VALUES(header_image),
    developer=VALUES(developer), publisher=VALUES(publisher), release_date=VALUES(release_date)
    """

    for i in range(0, len(results), 100):
        batch = results[i:i + 100]
        cursor.executemany(insert_query, batch)
        connection.commit()

    print("🎯 popular_games table populated successfully!")

# --------------------------------------------
# 6️⃣ Run (safe for Jupyter)
# --------------------------------------------
await main(limit=200000)

# Close DB
cursor.close()
connection.close()
print("✅ Done!")


✅ Total apps fetched from Steam: 273983


Fetching games: 100%|██████████| 200000/200000 [19:28<00:00, 171.12it/s] 


💾 Inserting 199 valid games into MySQL...
🎯 popular_games table populated successfully!
✅ Done!


In [4]:
import requests
import mysql.connector
from datetime import datetime

steam_api_key = "E579F61D0F6B642C45C82A7A946D5EF7"
steam_user_id = 76561199558514799

# Fetch all owned games including free ones
url = f"https://api.steampowered.com/IPlayerService/GetOwnedGames/v1/?key={steam_api_key}&steamid={steam_user_id}&include_appinfo=1&include_played_free_games=1"
response = requests.get(url).json()

owned_games = response['response'].get('games', [])
print(f"✅ Total owned games fetched: {len(owned_games)}")

# MySQL Connection
connection = mysql.connector.connect(
    host="localhost",
    user="python_user",
    password="aizen",
    database="game_recommender"
)
cursor = connection.cursor()

for game in owned_games:
    cursor.execute("""
        INSERT INTO user_library (user_id, appid, name, playtime_hours, added_at)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE playtime_hours=VALUES(playtime_hours)
    """, (
        1,  # example user_id
        game['appid'],
        game.get('name', ''),
        game.get('playtime_forever', 0)/60,  # convert minutes to hours
        datetime.now()
    ))

connection.commit()
cursor.close()
connection.close()
print(f"✅ {len(owned_games)} games added to user_library")


✅ Total owned games fetched: 11
✅ 11 games added to user_library


In [5]:
import mysql.connector
import pandas as pd

# Connect to MySQL
connection = mysql.connector.connect(
    host="localhost",
    user="python_user",
    password="aizen",
    database="game_recommender"
)

# Load user library
owned_df = pd.read_sql("SELECT * FROM user_library", connection)

# Load popular games
popular_games_df = pd.read_sql("SELECT * FROM popular_games", connection)

# Filter only owned games that exist in popular_games
owned_in_popular = owned_df[owned_df['name'].isin(popular_games_df['name'])]

print(f"Owned games present in popular_games: {len(owned_in_popular)}")

# Close connection
connection.close()


Owned games present in popular_games: 2


C:\Users\Yash\AppData\Local\Temp\ipykernel_27508\1313642386.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  owned_df = pd.read_sql("SELECT * FROM user_library", connection)
C:\Users\Yash\AppData\Local\Temp\ipykernel_27508\1313642386.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  popular_games_df = pd.read_sql("SELECT * FROM popular_games", connection)


In [14]:
missing_owned = owned_df[~owned_df['appid'].isin(popular_games_df['appid'])]
print(f"Missing owned games count: {len(missing_owned)}")
print(missing_owned[['appid', 'name']])


Missing owned games count: 9
      appid                                name
0     35140  Batman: Arkham Asylum GOTY Edition
1    200260            Batman: Arkham City GOTY
2    208650              Batman™: Arkham Knight
5   2073850                          THE FINALS
6   2231380              Ghost Recon Breakpoint
7   2567870                    Chained Together
8   2641470                      Spectre Divide
9   2767030                       Marvel Rivals
10  3028500              Cakey's Twisted Bakery


In [42]:
from fuzzywuzzy import process

game_name = "Counter-Strike: Global Offensive"
match = process.extractOne(game_name, popular_games_df['name'])
if match[1] > 80:  # similarity threshold
    user_game = match[0]


In [28]:
import pandas as pd
import mysql.connector

# 1️⃣ Connect to DB
connection = mysql.connector.connect(
    host="localhost",
    user="python_user",
    password="aizen",
    database="game_recommender"
)

# 2️⃣ Load data
user_library = pd.read_sql("SELECT * FROM user_library", connection)
popular_games = pd.read_sql("SELECT * FROM popular_games", connection)

# 3️⃣ Filter owned games that exist in popular_games using appid
owned_in_popular = user_library[user_library['appid'].isin(popular_games['appid'])]

print(f"✅ Owned games in popular games: {len(owned_in_popular)}")


✅ Owned games in popular games: 11


C:\Users\Yash\AppData\Local\Temp\ipykernel_27508\3549571812.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_library = pd.read_sql("SELECT * FROM user_library", connection)
C:\Users\Yash\AppData\Local\Temp\ipykernel_27508\3549571812.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  popular_games = pd.read_sql("SELECT * FROM popular_games", connection)


In [29]:
import mysql.connector
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Connect
connection = mysql.connector.connect(
    host="localhost",
    user="python_user",
    password="aizen",
    database="game_recommender",
    autocommit=True
)
cursor = connection.cursor(dictionary=True)

# Load user library
cursor.execute("SELECT appid, name FROM user_library WHERE user_id = 1")
user_library = pd.DataFrame(cursor.fetchall())

# Load popular games
cursor.execute("SELECT appid, name, genre, categories, short_des FROM popular_games")
popular_games = pd.DataFrame(cursor.fetchall())
cursor.close()
connection.close()


In [30]:
# Combine text features
popular_games['combined_features'] = (
    popular_games['genre'].fillna('') + ' ' +
    popular_games['categories'].fillna('') + ' ' +
    popular_games['short_des'].fillna('')
)

# TF-IDF matrix
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(popular_games['combined_features'])

# Map appid → index in TF-IDF
appid_to_index = {appid: idx for idx, appid in enumerate(popular_games['appid'])}


In [31]:
from collections import defaultdict
import heapq

top_n = 5  # top N recommendations per game
final_recommendations = defaultdict(list)

for _, owned in user_library.iterrows():
    appid = owned['appid']
    if appid not in appid_to_index:
        continue  # skip if owned game not in popular_games

    idx = appid_to_index[appid]
    similarity_scores = list(enumerate(cosine_similarity(tfidf_matrix[idx], tfidf_matrix)[0]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

    for i, score in similarity_scores[1:top_n+1]:  # skip self
        rec_name = popular_games.iloc[i]['name']
        final_recommendations[owned['name']].append((rec_name, score))


In [32]:
for owned_game, recs in final_recommendations.items():
    print(f"\nBecause you played '{owned_game}':")
    for rec_name, score in recs:
        print(f"  → {rec_name} (similarity: {score:.2f})")



Because you played 'Batman: Arkham Asylum GOTY Edition':
  → Batman: Arkham City - Game of the Year Edition (similarity: 0.30)
  → Batman™: Arkham Knight (similarity: 0.26)
  → Get Even - OST (similarity: 0.22)
  → RONIN - Special Edition Content (similarity: 0.18)
  → Dying Light (similarity: 0.16)

Because you played 'Batman: Arkham City GOTY':
  → Batman™: Arkham Knight (similarity: 0.32)
  → Batman: Arkham Asylum Game of the Year Edition (similarity: 0.30)
  → Get Even - OST (similarity: 0.26)
  → RONIN - Special Edition Content (similarity: 0.21)
  → Akash: Path of the Five Digital Deluxe Edition DLC (similarity: 0.20)

Because you played 'Batman™: Arkham Knight':
  → Batman: Arkham City - Game of the Year Edition (similarity: 0.32)
  → Batman: Arkham Asylum Game of the Year Edition (similarity: 0.26)
  → Borderlands 2 (similarity: 0.24)
  → Terraria (similarity: 0.24)
  → Life is Strange 2 - Japanese Language Pack (similarity: 0.21)

Because you played 'PUBG: BATTLEGROUNDS':
  →

In [33]:
import pandas as pd
import mysql.connector
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# --------------------------
# 1️⃣ Connect to MySQL
# --------------------------
connection = mysql.connector.connect(
    host="localhost",
    user="python_user",
    password="aizen",
    database="game_recommender"
)

# --------------------------
# 2️⃣ Load data
# --------------------------
user_library = pd.read_sql("SELECT appid, name FROM user_library WHERE user_id=1", connection)
popular_games = pd.read_sql("SELECT appid, name, genre, categories, short_des FROM popular_games", connection)

connection.close()

# --------------------------
# 3️⃣ Keep only owned games present in popular_games
# --------------------------
owned_in_popular = popular_games[popular_games['appid'].isin(user_library['appid'])]
print(f"✅ Owned games in popular games: {len(owned_in_popular)}")

# --------------------------
# 4️⃣ Prepare combined features
# --------------------------
popular_games['combined_features'] = (
    popular_games['genre'].fillna('') + ' ' +
    popular_games['categories'].fillna('') + ' ' +
    popular_games['short_des'].fillna('')
)

# --------------------------
# 5️⃣ TF-IDF vectorization
# --------------------------
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(popular_games['combined_features'])

# Mapping appid to index
appid_to_index = {appid: idx for idx, appid in enumerate(popular_games['appid'])}

# --------------------------
# 6️⃣ Compute recommendations
# --------------------------
top_n = 5  # top 5 similar games per owned game
recommendations = {}

for _, owned_game in tqdm(owned_in_popular.iterrows(), total=len(owned_in_popular), desc="Computing recommendations"):
    idx = appid_to_index[owned_game['appid']]
    similarity_scores = cosine_similarity(tfidf_matrix[idx], tfidf_matrix)[0]
    
    # Get top N indices excluding the owned game itself
    top_indices = similarity_scores.argsort()[::-1][1:top_n+1]
    recommended_games = [(popular_games.iloc[i]['name'], similarity_scores[i]) for i in top_indices]
    
    recommendations[owned_game['name']] = recommended_games

# --------------------------
# 7️⃣ Display recommendations
# --------------------------
for owned, recs in recommendations.items():
    print(f"\nBecause you played '{owned}':")
    for rec_name, score in recs:
        print(f"  → {rec_name} (similarity: {score:.2f})")


C:\Users\Yash\AppData\Local\Temp\ipykernel_27508\1167294301.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  user_library = pd.read_sql("SELECT appid, name FROM user_library WHERE user_id=1", connection)
C:\Users\Yash\AppData\Local\Temp\ipykernel_27508\1167294301.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  popular_games = pd.read_sql("SELECT appid, name, genre, categories, short_des FROM popular_games", connection)


✅ Owned games in popular games: 11


Computing recommendations: 100%|██████████| 11/11 [00:00<00:00, 827.77it/s]


Because you played 'Batman: Arkham Asylum Game of the Year Edition':
  → Batman: Arkham City - Game of the Year Edition (similarity: 0.30)
  → Batman™: Arkham Knight (similarity: 0.26)
  → Get Even - OST (similarity: 0.22)
  → RONIN - Special Edition Content (similarity: 0.18)
  → Dying Light (similarity: 0.16)

Because you played 'Batman: Arkham City - Game of the Year Edition':
  → Batman™: Arkham Knight (similarity: 0.32)
  → Batman: Arkham Asylum Game of the Year Edition (similarity: 0.30)
  → Get Even - OST (similarity: 0.26)
  → RONIN - Special Edition Content (similarity: 0.21)
  → Akash: Path of the Five Digital Deluxe Edition DLC (similarity: 0.20)

Because you played 'Batman™: Arkham Knight':
  → Batman: Arkham City - Game of the Year Edition (similarity: 0.32)
  → Batman: Arkham Asylum Game of the Year Edition (similarity: 0.26)
  → Borderlands 2 (similarity: 0.24)
  → Terraria (similarity: 0.24)
  → Life is Strange 2 - Japanese Language Pack (similarity: 0.21)

Because you

In [27]:
import requests
import mysql.connector

# MySQL connection
connection = mysql.connector.connect(
    host="localhost",
    user="python_user",
    password="aizen",
    database="game_recommender"
)
cursor = connection.cursor()

steam_api_key = "E579F61D0F6B642C45C82A7A946D5EF7"

missing_owned = [appid for appid in user_library['appid'] if appid not in popular_games['appid'].values]

for appid in missing_owned:
    url = f"https://store.steampowered.com/api/appdetails?appids={appid}"
    res = requests.get(url).json()
    data = res.get(str(appid), {}).get("data", None)
    if data:
        genres = ", ".join([g['description'] for g in data.get('genres', [])]) if data.get('genres') else ''
        categories = ", ".join([c['description'] for c in data.get('categories', [])]) if data.get('categories') else ''
        short_des = data.get('short_description', '')
        header_image = data.get('header_image', '')
        developer = ", ".join(data.get('developers', [])) if data.get('developers') else ''
        publisher = ", ".join(data.get('publishers', [])) if data.get('publishers') else ''
        release_date = data.get('release_date', {}).get('date', '')
        name = data.get('name', '')

        cursor.execute("""
            INSERT INTO popular_games (appid, name, genre, categories, short_des, header_image, developer, publisher, release_date)
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)
            ON DUPLICATE KEY UPDATE
            name=VALUES(name), genre=VALUES(genre), categories=VALUES(categories),
            short_des=VALUES(short_des), header_image=VALUES(header_image),
            developer=VALUES(developer), publisher=VALUES(publisher), release_date=VALUES(release_date)
        """, (appid, name, genres, categories, short_des, header_image, developer, publisher, release_date))

connection.commit()
cursor.close()
connection.close()
print(f"✅ Added {len(missing_owned)} missing owned games to popular_games")


✅ Added 9 missing owned games to popular_games
